Exercise 1: Assume we have $N$ individual and $J$ products in a market. $X_{ij}$ is the characteristic of product $j$ for individual $i$. $J_i$ is the choice of individual $i$ among the products. 

To generate the data, I assume $u_{ij} = X'_{j}\beta + \epsilon_{ij}$ where $X_j$ is created as independent standard normal random variables, $\epsilon_{ij}$ follows a T1EV, $\beta' = (5 , 2)$.

In [143]:
# See the below link for Distribution
# https://juliastats.github.io/Distributions.jl/stable/starting/
using LinearAlgebra
using Random, Distributions
using Optim

In [137]:
# Parameters

J = 5 #Number of products
N = 500 #Number of individuals
Jchar = 2 #Number of product characteristics
iter = 100 #Number of iterations
β = [5 ; 2];

In [146]:
# Data Generating Process dgp
function dgp(i)
    # Variables
    Random.seed!(100 + i) #Setting the seed
    X = rand(J, Jchar)
    return X
end

# Log Likelihood Function
function loglikelihood(β)
    P_nom = exp.( X*β )
    P = P_nom ./ sum( P_nom )
    L = sum( choice_mat' * log.(P) )
    
    return -L
end

loglikelihood (generic function with 1 method)

The probability that individual i chooses product j is
$$
P( i~chooses~j) = \frac{exp(\delta_j)}{\Sigma_{k \in J}~exp(\delta_k)}
$$
Therefore, the maximum likelihood estimator is
$$
L = \Pi_{i=1}^N\Pi_{j=1}^J~P(i->j)^{1(J_i == j)}
$$
and the log-likelihood function is
$$
logL = \Sigma_{i=1}^N\Sigma_{j=1}^J~1(J_i == j)\{log(P(i->j))\}
$$

I have to compute the log likelihood function for different value of $\beta$ which means I should multiply choice_mat with probs and take the summation and keep the max. Then repeat the whole process 100 times to find the average $\beta$ and its variance.

In [165]:
βhat = zeros(Jchar, iter)
for i in 1:iter
    
# Computing Share of Products
        X = dgp(i)
        δ = X*β
        d_2 = Gumbel() # or we can use d = Normal()
        Random.seed!(101 + i) #Setting the seed
        ϵ = rand(d_2, J , N)
        u = δ .+ ϵ
        u_max, ind = findmax(u, dims = 1)
        choice_mat = (u .>= u_max)
        s = sum( choice_mat, dims = 2)/N
    # Maximum Likelihood Estimation
        β_0 = ones(Jchar)
        optimum = optimize(loglikelihood, β_0)
        MLE = optimum.minimum
        βhat[ :, i] = optimum.minimizer
end
βhat

2×100 Array{Float64,2}:
 5.22646  5.31199  4.75858  4.63933  …  4.92204  4.36481  5.34319  5.06575
 2.1823   2.47085  1.95198  1.86305     2.09587  1.80447  2.14303  1.89103

In [171]:
println( mean( βhat , dims = 2 ) )
println( var( βhat , dims = 2) )

[5.092496672021896; 1.9994854737567205]
[0.460808291415688; 0.5728156849868485]
